In [ ]:
import __init__

#### 1st pass finetune - Mattergen XRD
- Dataset Source: [Mattergen Alex-MP-20](https://github.com/microsoft/mattergen/tree/main/data-release/alex-mp)
  - Columns: Database (manual) 
  - Reduced Formula (Source)
  - CIF (pmg - Cifwriter with symprec 0.1)
  - XRD 'Condition Vector' (with [_calculate_XRD.py](_utils/_preprocessing/_calculate_XRD.py))
    - pmg - XRDCalculator(wavelength="CuKa")
    - top 20 most intense peaks selected ($2\theta$ and int)
    - Normalisations
      - $2\theta$ min-max for 0,90
      - intensities min-max for 0,100
- Deduplicated
- Cleaned for CIF augmentation
  -  Note: I didnt filter to context length here because it was not implemented yet, but filter to context was flagged as True during model training which effectively does the same thing (less efficient)
- dataset pushed to HuggingFace as: c-bone/mattergen_XRD (90:10 train/valid sets)

In [ ]:
!torchrun --nproc_per_node=2 _train.py --config '_config_files/training/conditional/xrd_studies/mattergen_XRD-slider.jsonc'

#### 2nd pass finetune - COD XRD
- Dataset Source: [COD hkl data](https://www.crystallography.net/hkl/)
  - Columns: Database (manual) 
  - Reduced Formula (automated extraction from source)
  - CIF
    - automated extraction of material id from COD source
    - converted to structure using pmg COD.get_structure_by_id
    - Cifwriter with symprec 0.1 for CIF
    - note: this was done because alot of COD cifs arent in clean standard format. Pymatgen already did a big job of cleaning them up so we dont need to reinvent the wheel and take CIF data straight from source.
  - XRD data
    - For every Material ID that has experimental hkl data and associated intensities, we extract it
    - Then:
      1. Calculate d_hkl from crystal structure.lattice.d_hkl([h, k, l])
      2. Use Bragg's law: sin($\theta$) = $\lambda$/($2$ × d_hkl)
      3. Find $\theta$ = arcsin($\lambda$/($2$ × d_hkl))
      4. Convert to degrees: $2\theta$ = $2$ × $\theta$ × (180/$\pi$)
    - Where:
      - $\lambda$: X-ray wavelength ($1.5406$ $\AA$ for Cu K$\alpha$)
      - d_hkl: d-spacing for the (hkl) planes
      - $\theta$: Bragg angle
    - Created 'Condition Vector'
      - top 20 most intense peaks selected ($2\theta$ and int)
      - Normalisations
        - $2\theta$ min-max for 0,90
        - intensities min-max for 0,100
  - Filtered out all hydrocarbons
    - symbols = struct.symbol_set
    - if "C" in symbols and "H" in symbols, remove it
  - Then cleaning for CIF augmentation
    - set --make_disordered_ordered flag
      - Makes every occupancy exactly integer if occupancy is int $\pm 0.05$. Element set needs to be exactly preserved or structure discarded.
    - Filtered to 1024 contect length
  - Pushed to HuggingFace as c-bone/COD_XRD_small_nohc

### Training
> **Note**: Here the hyperparamters change compared to regular finetuning because its 2nd pass. Backbone learning rates were set to decay from $5\times10^{-8}$ to $5\times10^{-10}$, while the learning rates for the newly initialised conditioning parameters were set 100 times higher

In [ ]:
!python _utils/_preprocessing/_save_dataset_to_HF.py \
    --input_parquet 'HF-databases/COD_dev/COD_xrd_clean_nohc_small.parquet' \
    --output_parquet 'HF-databases/COD_XRD_small_nohc_full.parquet' \
    --valid_size 0.000 \
    --test_size 0.125 \
    --save_hub

In [ ]:
!torchrun --nproc_per_node=2 _train.py --config '_config_files/training/conditional/xrd_studies/COD_XRD_small-slider-opt.jsonc'

In [ ]:
# Create reference file from test split
import pandas as pd
df = pd.read_parquet('HF-databases/COD_XRD_small_nohc_full.parquet')
df_test = df[df['Split'] == 'test'].copy()
df_test.to_parquet('_artifacts/cod-xrd/cod-test_ref.parquet', index=False)

### Generating

In [ ]:
!python _utils/_generating/make_prompts.py \
    --HF_dataset 'c-bone/COD_XRD_small_nohc' \
    --split 'test' \
    --automatic \
    --output_parquet '_artifacts/cod-xrd/cod-test_prompts.parquet' \
    --level 'level_3' \
    --condition_columns 'Condition Vector'

#### Generate materials using 2-pass finetuning and XRD information (Repeated 3x)

In [ ]:
import __init__

In [ ]:
!python _utils/_generating/generate_CIFs.py --config '_config_files/generation/conditional/xrd_studies/cod-xrd_eval.jsonc'

In [ ]:
!python _utils/_generating/postprocess.py \
    --input_parquet '_artifacts/cod-xrd/cod-ft-20perp-test_gen.parquet' \
    --output_parquet '_artifacts/cod-xrd/cod-ft-20perp-test_post.parquet' \
    --num_workers 32 \
    --column_name 'Generated CIF'

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/cod-xrd/cod-ft-20perp-test_post.parquet' \
    --num_gens 20 \
    --ref_parquet '_artifacts/cod-xrd/cod-test_ref.parquet' \
    --output_parquet '_artifacts/cod-xrd/cod-ft-20perp-test_metrics.parquet' \
    --num_workers 32 \
    --validity_check "none"

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/cod-xrd/cod-ft-20perp-test_post.parquet' \
    --num_gens 1 \
    --ref_parquet '_artifacts/cod-xrd/cod-test_ref.parquet' \
    --output_parquet '_artifacts/cod-xrd/cod-ft-1perp-test_metrics.parquet' \
    --num_workers 32 \
    --validity_check "none"

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/cod-xrd/cod-ft-20perp-test_post.parquet' \
    --num_gens 1 \
    --ref_parquet '_artifacts/cod-xrd/cod-test_ref.parquet' \
    --output_parquet '_artifacts/cod-xrd/cod-ft-1rand-test_metrics.parquet' \
    --num_workers 32 \
    --validity_check "none"\
    --sort_gens "random"

#### Generate materials using 2-pass finetuning (Mattergen XRD + COD XRD nohc) but no XRD information fed during inference (repeated 3x)

> **Note**: replaced the condition_vector column in the prompt df made above with a series of [-100] missing values meaning no XRD information is fed during generation

In [ ]:
import __init__

In [ ]:
!python _utils/_generating/generate_CIFs.py --config '_config_files/generation/conditional/xrd_studies/cod-xrd-uncond_eval.jsonc'

In [ ]:
!python _utils/_generating/postprocess.py \
    --input_parquet '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-20perp-test_gen.parquet' \
    --output_parquet '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-20perp-test_post.parquet' \
    --num_workers 32 \
    --column_name 'Generated CIF'

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-20perp-test_post.parquet' \
    --num_gens 20 \
    --ref_parquet '_artifacts/cod-xrd/perp-repeats/cod-test_ref.parquet' \
    --output_parquet '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-20perp-test_metrics.parquet' \
    --num_workers 32 \
    --validity_check "none"

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-20perp-test_post.parquet' \
    --num_gens 1 \
    --ref_parquet '_artifacts/cod-xrd/perp-repeats/cod-test_ref.parquet' \
    --output_parquet '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-1perp-test_metrics.parquet' \
    --num_workers 32 \
    --validity_check "none"

In [ ]:
metrics_1perp_parquet = '_artifacts/cod-xrd/perp-repeats/cod-ft-1perp-test_metrics.parquet'
metrics_1perp_v2_parquet= '_artifacts/cod-xrd/perp-repeats/cod-ft-1perp-v2-test_metrics.parquet'
metrics_1perp_v3_parquet= '_artifacts/cod-xrd/perp-repeats/cod-ft-1perp-v3-test_metrics.parquet'
metrics_20perp_parquet = '_artifacts/cod-xrd/perp-repeats/cod-ft-20perp-test_metrics.parquet'
metrics_20perp_v2_parquet = '_artifacts/cod-xrd/perp-repeats/cod-ft-20perp-v2-test_metrics.parquet'
metrics_20perp_v3_parquet = '_artifacts/cod-xrd/perp-repeats/cod-ft-20perp-v3-test_metrics.parquet'
metrics_uncond_1perp_parquet = '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-1perp-test_metrics.parquet'
metrics_uncond_1perp_v2_parquet = '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-1perp-v2-test_metrics.parquet'
metrics_uncond_1perp_v3_parquet = '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-1perp-v3-test_metrics.parquet'
metrics_uncond_20perp_parquet = '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-20perp-test_metrics.parquet'
metrics_uncond_20perp_v2_parquet = '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-20perp-v2-test_metrics.parquet'
metrics_uncond_20perp_v3_parquet = '_artifacts/cod-xrd/perp-repeats/cod-ft-uncond-20perp-v3-test_metrics.parquet'


# make a table with all the results
import __init__
from _utils import get_metrics_xrd
import pandas as pd
import numpy as np

paths = {
    'cond-20perp': metrics_20perp_parquet,
    'cond-20perp-v2': metrics_20perp_v2_parquet,
    'cond-20perp-v3': metrics_20perp_v3_parquet,
    'cond-1perp': metrics_1perp_parquet,
    'cond-1perp-v2': metrics_1perp_v2_parquet,
    'cond-1perp-v3': metrics_1perp_v3_parquet,
    'uncond-20perp': metrics_uncond_20perp_parquet,
    'uncond-20perp-v2': metrics_uncond_20perp_v2_parquet,
    'uncond-20perp-v3': metrics_uncond_20perp_v3_parquet,
    'uncond-1perp': metrics_uncond_1perp_parquet,
    'uncond-1perp-v2': metrics_uncond_1perp_v2_parquet,
    'uncond-1perp-v3': metrics_uncond_1perp_v3_parquet
}
results = {}

for names, path in paths.items():
    df = pd.read_parquet(path)
    metrics_result = get_metrics_xrd(df, n_test=198, only_matched=False, verbose=False)
    results[names] = metrics_result
        
# Create final table with all results
final_table = pd.DataFrame.from_dict(results, orient='index')
final_table.to_parquet('_artifacts/cod-xrd/cod-ft-vs-uncond-all-results.parquet')

In [ ]:
import __init__
import pandas as pd
import numpy as np

final_table = pd.read_parquet('_artifacts/cod-xrd/cod-ft-vs-uncond-all-results.parquet')

base_conditions = {}
for index in final_table.index:
    if '-v2' in index:
        base_name = index.replace('-v2', '')
    elif '-v3' in index:
        base_name = index.replace('-v3', '')
    else:
        base_name = index
    
    if base_name not in base_conditions:
        base_conditions[base_name] = []
    base_conditions[base_name].append(index)

# averaged results with standard error between the 3 runs
averaged_results = {}
stderr_results = {}
for base_name, variants in base_conditions.items():
    variant_data = []
    for variant in variants:
        if variant in final_table.index:
            variant_data.append(final_table.loc[variant])
    
    if variant_data:
        data_df = pd.concat(variant_data, axis=1)
        averaged_results[base_name] = data_df.mean(axis=1)
        stderr_results[base_name] = data_df.std(axis=1) / np.sqrt(len(variant_data))

averaged_table = pd.DataFrame.from_dict(averaged_results, orient='index')
stderr_table = pd.DataFrame.from_dict(stderr_results, orient='index')

formatted_table = pd.DataFrame(index=averaged_table.index, columns=averaged_table.columns, dtype=object)
for col in averaged_table.columns:
    for idx in averaged_table.index:
        mean_val = averaged_table.loc[idx, col]
        stderr_val = stderr_table.loc[idx, col]
        if pd.isna(stderr_val) or stderr_val == 0:
            formatted_table.loc[idx, col] = f"{mean_val:.3f}"
        else:
            formatted_table.loc[idx, col] = f"{mean_val:.3f} (±{stderr_val:.3f})"

formatted_table

> Note: the results here differ very slightly from the plots in the paper, because those plots calculate R^2 and MAE over all of the rows in the 3 runs (concatenated results), whereas here its the average between the results of each run as a whole and the associated stderr

### Testing on some real data

- Had the chance to get given some XRD data calculated by a group. It was calculated for brookite, anatase and rutiile poolymorphs of TiO2
- Anatase and rutile were seen during training and finetuning (in pretrain data and the mattergen xrd 1st pass finetune dataset), brookite was not
- Can the model generate the correct structures for experimental XRDs for materials seen in training, and one unseen?

1. First we make a dataset with the true structures as per their materials project structures
2. To this we add a prompt for each of the structures
3. And a condition vector as per below

In [ ]:
import __init__
from _utils import process_xrd_to_condition_vector

anatase_raw_data = """2θ [°] Cu	Intensity
25.281 100.0
36.947 10.0
37.801 20.0
38.576 10.0
48.050 35.0
53.891 20.0
55.062 20.0
62.121 4.0
62.690 14.0
68.762 6.0
70.311 6.0
74.031 2.0
75.032 10.0
76.020 4.0
80.727 2.0
82.139 2.0
82.662 6.0
83.149 4.0
83.221 2.0
"""

rutile_raw_data = """2θ [°] Cu	Intensity
27.447 100.0
36.086 50.0
39.188 8.0
41.226 25.0
44.052 10.0
54.323 60.0
56.642 20.0
62.742 10.0
64.040 10.0
65.480 2.0
69.010 20.0
69.790 12.0
72.410 2.0
74.411 1.0
76.510 4.0
79.822 2.0
82.335 6.0
84.260 4.0
87.464 2.0
89.557 8.0"""

brookite_raw_data = """2θ [°] Cu	Intensity
25.35 100.0
25.69 77.71
30.81 97.23
36.24 23.86
37.28 17.77
37.96 7.41
40.16 14.94
42.33 15.5
46.07 19.95
48.03 32.31
49.16 20.2
54.22 22.63
55.24 31.92
57.14 15.52
62.08 11.59
63.41 7.68
63.69 10.48
64.98 13.99
65.94 8.26
70.44 9.04
"""


# Test the function
anatase = process_xrd_to_condition_vector(anatase_raw_data)
rutile = process_xrd_to_condition_vector(rutile_raw_data)
brookite = process_xrd_to_condition_vector(brookite_raw_data)

print(anatase)
print(rutile)
print(brookite)


In [ ]:
import pandas as pd
df_path = '_artifacts/cod-xrd/amil/amil-TiO2-nosc-nosg_ref_prompts.parquet'

df = pd.read_parquet(df_path)
df

> Note: I added the condition vector to a dataframe where I stored prompts,true CIF of material, and some information for the experiment. We can use this as both prompt and ref dfs because it contains all the relevant columns

Here we want to probe a few things, can we recover structues from XRD data sent to us by a group we are collaborating with? for 3 different XRD polymorphs. 2 of them were seen in training sets, though neither were in the experimental finetuning set (rutile, anatase), and one of them hasnt been seen at all (training or finetuning).

We want to see if we can recover the structures from various possible experimental (their composition), sometimes we have their spacegroup (comp + spacegroup), sometimes we dont really know the composition, so is the model able to match to the material when prompted with the composition of its supercell?

In [ ]:
import pandas as pd
import __init__

In [ ]:
# Loop through each of the 4 split files
prompt_files = [
    'amil-TiO2-nosc-nosg_ref_prompts.parquet',
    'amil-TiO2-sc-nosg_ref_prompts.parquet', 
    'amil-TiO2-nosc-sg_ref_prompts.parquet',
    'amil-TiO2-sc-sg_ref_prompts.parquet'
]

config_files = [
    '_config_files/generation/conditional/xrd_studies/amil/cod-amil-xrd-nosc-nosg_eval.jsonc',
    '_config_files/generation/conditional/xrd_studies/amil/cod-amil-xrd-sc-nosg_eval.jsonc',
    '_config_files/generation/conditional/xrd_studies/amil/cod-amil-xrd-nosc-sg_eval.jsonc', 
    '_config_files/generation/conditional/xrd_studies/amil/cod-amil-xrd-sc-sg_eval.jsonc'
]

output_gen_files = [
    'amil-TiO2-nosc-nosg_gen.parquet',
    'amil-TiO2-sc-nosg_gen.parquet',
    'amil-TiO2-nosc-sg_gen.parquet',
    'amil-TiO2-sc-sg_gen.parquet'
]
output_metrics_files = [
    'amil-TiO2-nosc-nosg_metrics.parquet',
    'amil-TiO2-sc-nosg_metrics.parquet',
    'amil-TiO2-nosc-sg_metrics.parquet',
    'amil-TiO2-sc-sg_metrics.parquet'
]

In [ ]:

for i, (prompt_file, config_file, output_file) in enumerate(zip(prompt_files, config_files, output_gen_files)):
    print(f"\nProcessing {i+1}/4: {prompt_file}")
    
    # Generate CIFs
    !python _utils/_generating/generate_CIFs.py --config '{config_file}'
    
    # Postprocess
    !python _utils/_generating/postprocess.py \
        --input_parquet '_artifacts/cod-xrd/amil/{output_file}' \
        --output_parquet '_artifacts/cod-xrd/amil/{output_file}' \
        --num_workers 32 \
        --column_name 'Generated CIF'

print("\nGeneration and postprocessing complete for all files!")

In [ ]:
print("Running XRD metrics for all 4 generated files...")


for i, (gen_file, prompt_file, metrics_file) in enumerate(zip(output_gen_files, prompt_files, output_metrics_files)):
    print(f"\nCalculating metrics for {i+1}/4: {gen_file}")
    
    !python _utils/_metrics/XRD_metrics.py \
        --input_parquet '_artifacts/cod-xrd/amil/{gen_file}' \
        --num_gens 20 \
        --ref_parquet '_artifacts/cod-xrd/amil/{prompt_file}' \
        --output_parquet '_artifacts/cod-xrd/amil/{metrics_file}' \
        --num_workers 4 \
        --validity_check "none"

print("\nXRD metrics calculation complete for all files!")

In [ ]:
# for all the 4 files, make a table with all the results
import pandas as pd
import __init__
from _utils import get_metrics_xrd

output_metrics_files = [
    'amil-TiO2-nosc-nosg_metrics.parquet',
    'amil-TiO2-sc-nosg_metrics.parquet',
    'amil-TiO2-nosc-sg_metrics.parquet',
    'amil-TiO2-sc-sg_metrics.parquet'
]

# concatenate all the metrics files into one dataframe
all_metrics = pd.concat([pd.read_parquet(f'_artifacts/cod-xrd/amil/{f}') for f in output_metrics_files], ignore_index=True)
all_metrics
